In [91]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from math import *
import csv
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from pandas import concat
from pandas import DataFrame

from math import sqrt
from numpy import concatenate


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [92]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [93]:
dataPath = "project\data\ME_data_new.csv"
data = pd.read_csv(dataPath)
data=data.iloc[:,2:7]
encoder1 = LabelEncoder()
values1=data.iloc[:,2]
values1 = encoder1.fit_transform(values1)
# values1 = encoder1.inverse_transform(values1)
encoder2 = LabelEncoder()
values2=data.iloc[:,4]
values2 = encoder2.fit_transform(values2)
# values2 = encoder2.inverse_transform(values2)
data['WindDirection']=values1
data['Weather']=values2
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(data)
reframed = series_to_supervised(scaled, 1, 1)
reframed

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var1(t),var2(t),var3(t),var4(t),var5(t)
1,0.162162,0.15625,1.000,0.223203,0.923077,0.108108,0.21875,1.000,0.384989,0.907692
2,0.108108,0.21875,1.000,0.384989,0.907692,0.135135,0.18750,1.000,0.094220,0.184615
3,0.135135,0.18750,1.000,0.094220,0.184615,0.243243,0.18750,1.000,0.224111,0.184615
4,0.243243,0.18750,1.000,0.224111,0.184615,0.162162,0.21875,1.000,0.357837,0.184615
5,0.162162,0.21875,1.000,0.357837,0.184615,0.108108,0.21875,1.000,0.090351,0.184615
...,...,...,...,...,...,...,...,...,...,...
2793,0.459459,0.62500,0.250,0.100178,0.538462,0.513514,0.56250,0.500,0.314114,0.538462
2794,0.513514,0.56250,0.500,0.314114,0.538462,0.432432,0.62500,0.125,0.154660,0.538462
2795,0.432432,0.62500,0.125,0.154660,0.538462,0.486486,0.62500,0.375,0.103270,0.538462
2796,0.486486,0.62500,0.375,0.103270,0.538462,0.729730,0.71875,0.250,0.055836,0.538462


In [94]:
values = reframed.values
n_train_hours = 2000
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-5], train[:, -5:]  
test_X, test_y = test[:, :-5], test[:, -5:]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(2000, 1, 5) (2000, 5) (797, 1, 5) (797, 5)


In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(5))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=50, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
yhat = scaler.inverse_transform(yhat)
# calculate RMSE
rmse = sqrt(mean_squared_error(test_X, yhat))
print('Test RMSE: %.3f' % rmse)